# Text mining jaarverslagen
1. Read info from the `file_list.csv`.
2. Uses `analyse_jaarverslag.ipynb` to analyse each jaarverslag, one at a time.

## 1. Read from `file_list.csv`

In [1]:
from os.path import join, isfile
import pandas as pd

folder = '../jaarverslagen'
files = pd.read_csv(join(folder, 'file_list.csv'))
files

,filename,language
0,ABNAMRO_2017.pdf,english
1,AEGON_2017.pdf,english
2,Akzonobel_2017.pdf,english
3,Heineken_2017.pdf,english
4,ING_Groep_2017.pdf,english
5,KPN_2017.pdf,english
6,Philips_2017.pdf,english
7,Unilever_2017.pdf,english


## 2. Run through all jaarverslagen
Using `papermill`, see https://papermill.readthedocs.io/en/latest/usage.html.

How does this work with my virtual environment `tmj`?
* Jupyter Notebook is served from `base` environment which has `widgetnbextension` enabled
* Next `ipywidgets` is installed in this kernel's environment `tmj`
* See https://ipywidgets.readthedocs.io/en/stable/user_install.html#installing-with-multiple-environments for more info about this

In [2]:
import papermill as pm

We'll execute the notebook `analyse_jaarverslag` and have the same output file as the input file. This means that the parameters will be inserted (`injected-paramaters` cell) into the original notebook.

In [3]:
# Loop over all files
for row in files.itertuples():
    
    #
    print('Running:', row.filename)
    
    # First check if it exists
    if not isfile(join(folder, row.filename)):
        print('File not found:', row.filename)
        continue
    
    # Execute the notebook
    pm.execute_notebook(
       'analyse_jaarverslag.ipynb',
       'analyse_jaarverslag.ipynb',
       parameters = dict(filename = row.filename,
                         folder = folder,
                         language = row.language)
    )

Running: ABNAMRO_2017.pdf


KeyboardInterrupt: 

## 3. Extract the data from the output folders
And combine it of course.

Each output folder corresponds to 1 jaarverslag and includes multiple CSV-PNG couples. Each CSV (table with unique words and their frequencies) and PNG (accompanying word cloud) have a *tag* which describes the current stage in the text mining process.

In [10]:
# Let's stick to one tag for now
tag = '_basic_processing'

# Our output folder
output_folder = '../output/'

# Maximum words (most common) to extract from the files
max_rows = 1000
max_rows = None # For IDF we need no limit

In [11]:
# Create a list (of tuples) with the path and filename of our CSVs
from os import walk
import re
csvs = []

# Walk over the entire contents of the output folder
for path, _, files in walk(output_folder):
    # But skip the folder itself, only look into subfolders
    if path == output_folder:
        continue
    
    # Loop over each file in the folder
    for filename in files:
        # Only store the CSVs we're interested in
        if re.match('.+' + tag + '\.csv', filename):
            csvs.append((path, filename))

csvs

[('../output/ABNAMRO_2017', 'ABNAMRO_2017_basic_processing.csv'),
 ('../output/AEGON_2017', 'AEGON_2017_basic_processing.csv'),
 ('../output/Akzonobel_2017', 'Akzonobel_2017_basic_processing.csv'),
 ('../output/Heineken_2017', 'Heineken_2017_basic_processing.csv'),
 ('../output/ING_Groep_2017', 'ING_Groep_2017_basic_processing.csv'),
 ('../output/KPN_2017', 'KPN_2017_basic_processing.csv'),
 ('../output/Philips_2017', 'Philips_2017_basic_processing.csv'),
 ('../output/Unilever_2017', 'Unilever_2017_basic_processing.csv')]

Now we have the files we need. Read them into a large data frame `df_tf`.

And calculate the term frequency `tf` which is the count normalized by the total number of words.

In [21]:
import pandas as pd

# Initiate all results data frame
df_tf = pd.DataFrame()

# Loop over our files
for (path, filename) in csvs:
    # Pandas read
    df = pd.read_csv(join(path, filename), header=None, names=['word', 'count'],
                     #nrows=max_rows # For debugging only
                    )

    # Sort (again, but just to make sure)
    df.sort_values(by='count', ascending=False, inplace=True)
    
    # Calculate the term frequency: divide by the total amount of words in the document
    total_amount = df['count'].sum()
    df['tf'] = df['count'] / total_amount

    # Truncate to maximum rows
    df = df[:max_rows]

    # Add the report name (filename without tag) as a column
    df['filename'] = re.match('(.+)' + tag + '\.csv', filename).group(1)

    # Append to rest of results
    df_tf = df_tf.append(df)
    print(df_tf.shape)

df_tf.head()

(5679, 4)
(10019, 4)
(15779, 4)
(20906, 4)
(28503, 4)
(34109, 4)
(40729, 4)
(48001, 4)


,word,count,tf,filename
0,risk,1551,0.018978,ABNAMRO_2017
1,abn,1352,0.016543,ABNAMRO_2017
2,amro,1352,0.016543,ABNAMRO_2017
3,financial,1269,0.015527,ABNAMRO_2017
4,annual,1014,0.012407,ABNAMRO_2017


In [22]:
# Save as CSV
df_tf.to_csv(join(output_folder, 'Alle_jaarverslagen' + tag + '.csv'),
             index=False
            )

## 4. Inverse document frequency

IDF is the inverse function of the amount of documents the term occurs in. We define it as:

$$\text{idf}(t,D)=\log{ \frac{N}{|\{d \in D:t \in d\}|} }$$

where $t$ is a term or word, $D$ is the collection of all documents or corpus, $N$ is the total amount of documents and $d$ is one document. $|\{d \in D:t \in d\}|$ means the sum of all documents $d$ in $D$ which have term $t$ in their body. Note that $\log$ is the natural logarithm.

See https://en.wikipedia.org/wiki/tf-idf for more info.

*Note:* Maybe this will clear to many words? Maybe take into account the counts of other documents? For example: the word 'risk' is in every document. But perhaps in 1 document way more than in others.

In [24]:
# Extract and count unique filenames in the data frame
N = len(set(df_tf['filename']))

# Group by word and count the amount of filenames
idf = df_tf.groupby(by='word', as_index=False)['filename'].count().rename(columns={'filename': 'document_count'})

# Calculate IDF
import numpy as np
idf['idf'] = np.log(N/idf['document_count'])
idf.head()

,word,document_count,idf
0,aa,7,0.133531
1,aaa,3,0.980829
2,aaarated,3,0.980829
3,aag,1,2.079442
4,aaidea,1,2.079442


### Join TF list with IDF list
The IDF data frame is word specific and document (jaarverslag) inspecific. The TF data frame contains words (and count) per document.

So join TF with IDF and calculate TF-IDF.

In [35]:
# Join TF with IDF
df_tfidf = df_tf.merge(idf, how='left', on='word')

# Calculate TF-IDF
df_tfidf['tf-idf'] = df_tfidf['tf'] * df_tfidf['idf']

# Sort
df_tfidf.sort_values(by=['filename', 'tf-idf'], ascending=[True, False], inplace=True)
df_tfidf.head(10)

,word,count,tf,filename,document_count,idf,tf-idf
1,abn,1352,0.016543,ABNAMRO_2017,5.0,0.470004,0.007775
2,amro,1352,0.016543,ABNAMRO_2017,5.0,0.470004,0.007775
18,banking,469,0.005739,ABNAMRO_2017,4.0,0.693147,0.003978
230,nlfi,63,0.000771,ABNAMRO_2017,1.0,2.079442,0.001603
241,stak,61,0.000746,ABNAMRO_2017,1.0,2.079442,0.001552
245,aag,61,0.000746,ABNAMRO_2017,1.0,2.079442,0.001552
6,eur,892,0.010914,ABNAMRO_2017,7.0,0.133531,0.001457
264,rea,57,0.000697,ABNAMRO_2017,1.0,2.079442,0.001450
105,mortgages,115,0.001407,ABNAMRO_2017,3.0,0.980829,0.001380
174,forborne,76,0.000930,ABNAMRO_2017,2.0,1.386294,0.001289


In [36]:
# Save as CSV
columns = ['word', 'count', 'filename', 'tf', 'tf-idf']
df_tfidf[columns].to_csv(
    join(output_folder, 'Alle_jaarverslagen' + tag + '.csv'),
    index=False
)